# Run points THDM-II model

The idea is run the model

In [1]:
import numpy as np
import pandas as pd
import subprocess
import time

%matplotlib inline
import matplotlib.pyplot as plt 

Input file:

In [2]:
#xd0 = pd.read_csv('L4_Negative_tb_2073.txt')
xd0 = pd.read_csv('L4_Positive_tb_2073.txt')

In [6]:
xd0[:2]

,L1,L2,L3,L4,L5,M12,tb
0,0.161231,0.30164,0.125743,-0.357056,-0.21088,-107152.0,2.07321
1,0.161231,0.30164,0.125743,-0.357056,-0.21088,-109648.0,2.07321


In [7]:
xd0.shape[0]

507

In [8]:
import pyslha
import pyTHDM_LesHouches_generator
import sys
import time

In [9]:
# Tiempo inicial
t1=time.time() 

x=[]

#Change
#Num = 10
Num = xd0.shape[0]

for i in range (0,Num):
       
    L1 = xd0['L1'][i] 
    L2 = xd0['L2'][i]
    L3 = xd0['L3'][i]
    L4 = xd0['L4'][i]
    L5 = xd0['L5'][i]
    M12 = xd0['M12'][i]
    tb = 2.07321

    #Open the file that will created the LesHouches file
    xdict = pyTHDM_LesHouches_generator.buildSLHAinFile()

    xdict.blocks['MINPAR'].entries[1]='%.6E    # Lambda1Input'%L1
    xdict.blocks['MINPAR'].entries[2]='%.6E    # Lambda2Input'%L2
    xdict.blocks['MINPAR'].entries[3]='%.6E    # Lambda3Input'%L3
    xdict.blocks['MINPAR'].entries[4]='%.6E    # Lambda4Input'%L4
    xdict.blocks['MINPAR'].entries[5]='%.6E    # Lambda5Input'%L5
    xdict.blocks['MINPAR'].entries[9]='%.6E    # M12inputt'%M12
    xdict.blocks['MINPAR'].entries[10]='%.6E    # TanBeta' %tb

    #Write and created the Leshouches file generator and closed
    pyslha.writeSLHAFile('LesHouches.in.THDMII_low',xdict)  

    #run SPheno
    spheno = subprocess.getoutput('~/Work/SPheno-4.0.4/bin/SPhenoTHDMII LesHouches.in.THDMII_low')    
    SPheno_output = subprocess.getoutput('cat SPheno.spc.THDMII')
    
    #capturo algo de spheno
    #MASAS
    mh1 = eval(SPheno_output.split('#   PDG code')[1].split()[3])
    mh2 = eval(SPheno_output.split('#   PDG code')[1].split()[7])
    mAh = eval(SPheno_output.split('#   PDG code')[1].split()[11])
    mHm = eval(SPheno_output.split('#   PDG code')[1].split()[15])
    #print("========== MASAS ==============")
    #print("mh1=",mh1,"mh2=",mh2,"mAh=",mAh,"mHm=",mHm)
    #print("===============================\n")

    #Couplings
    h1bb = eval(SPheno_output.split('HiggsCouplingsFermions')[1].split()[1])
    h1tt = eval(SPheno_output.split('# h_1 s s coupling')[1].split()[0])
    h1tautau = eval(SPheno_output.split('# h_1 c c coupling')[1].split()[0])
    h1VV = eval(SPheno_output.split('HiggsCouplingsBosons')[1].split()[1])
    
    #print("============ Couplings ================")
    #print("chbb=",h1bb,"chtt=",h1tt,"chtautau=",h1tautau,"hVV=",h1VV)
    #print("============================\n")
    
    #RUN Higgsbounds
    
    #copio los archivos a la carpeta Input_files
    subprocess.getoutput('cp Messages.out effC.dat BR* Key.dat LesHouches.in.THDMII_low MH* SPheno.spc.THDMII Input_Files/')
    #run HiggsBounds
    hb = subprocess.getoutput('~/Work/HiggsBounds-5.3.2beta/./HiggsBounds LandH effC 3 1 ~/Work/Documents_compartidos/THDM-II/Input_Files/')
    
    #Capturo lo que necesito del archivo de resultados
    HB_results = subprocess.getoutput('cat Input_Files/HiggsBounds_results.dat')
    HBresult = eval(HB_results.split('cols:')[1].split()[15])
    #print("HBresult=",HBresult)
    
    x.append([L1,L2,L3,L4,L5,M12,tb,mh1,mh2,mAh,mHm,h1bb,h1tt,h1tautau,h1VV,HBresult])

x=np.asarray(x)
#Write the output
xd=pd.DataFrame(x,columns=['L1','L2','L3','L4','L5','M12','tb','mh1','mh2','mAh','mHm','h1bb','h1tt','h1tautau','h1VV','HBresult'])  

t2=time.time() 
print ("The program spent", t2-t1, "s running",Num,"times")   

The program spent 1005.4190645217896 s running 507 times


In [12]:
1005.17/60

16.75283333333333

In [10]:
xd[:2]

,L1,L2,L3,L4,L5,M12,tb,mh1,mh2,mAh,mHm,h1bb,h1tt,h1tautau,h1VV,HBresult
0,0.161231,0.30164,0.125743,-0.357056,-0.21088,-107152.0,2.07321,124.675187,540.045492,535.366426,539.488775,1.123765,0.968941,1.123765,0.998163,1.0
1,0.161231,0.30164,0.125743,-0.357056,-0.21088,-109648.0,2.07321,124.766931,545.898260,541.290932,545.368500,1.121020,0.969681,1.121020,0.998245,1.0


save output file:

In [11]:
#xd.to_csv('L4_Negative_tb_2073_output.csv')
#xd.to_csv('L4_Positive_tb_2073_output.csv')

In [14]:
xd0.shape

(507, 7)